# NVIDIA AI Foundation Endpoints

The `ChatNVIDIA` class is a LangChain chat model that connects to [NVIDIA AI Foundation Endpoints](https://www.nvidia.com/en-us/ai-data-science/foundation-models/).


> [NVIDIA AI Foundation Endpoints](https://www.nvidia.com/en-us/ai-data-science/foundation-models/) give users easy access to NVIDIA hosted API endpoints for NVIDIA AI Foundation Models like Mixtral 8x7B, Llama 2, Stable Diffusion, etc. These models, hosted on the [NVIDIA NGC catalog](https://catalog.ngc.nvidia.com/ai-foundation-models), are optimized, tested, and hosted on the NVIDIA AI platform, making them fast and easy to evaluate, further customize, and seamlessly run at peak performance on any accelerated stack.
> 
> With [NVIDIA AI Foundation Endpoints](https://www.nvidia.com/en-us/ai-data-science/foundation-models/), you can get quick results from a fully accelerated stack running on [NVIDIA DGX Cloud](https://www.nvidia.com/en-us/data-center/dgx-cloud/). Once customized, these models can be deployed anywhere with enterprise-grade security, stability, and support using [NVIDIA AI Enterprise](https://www.nvidia.com/en-us/data-center/products/ai-enterprise/).
> 
> These models can be easily accessed via the [`langchain-nvidia-ai-endpoints`](https://pypi.org/project/langchain-nvidia-ai-endpoints/) package, as shown below.

This example goes over how to use NVIDIA Inference Microservices (NIM)'s reranker, to spin up a reranker service as an API call( i.e obtaining an URL) , we will then combined this with NVIDIA AI Foundation endpoints' models to establish a pipeline.

## Setup

The integration lives in the `langchain` package. We also need to install the `langchain-nvidia-ai-endpoints` and `faiss` package itself. We can install these with:

```bash
pip install --quiet  langchain-nvidia-ai-endpoints
# depending the underlying compute resource, you can choose to install faiss-cpu or faiss-gpu 
pip install --quiet faiss-cpu/gpu
```

We'll also need to spin up and get an API call from [NVIDIA Inference Microservice(NIM) - Reranker ](https://nvidia-micro-service.com/) and set the `NVReranker_URL` as environment variable

In [1]:
!pip install langchain-nvidia-ai-endpoints

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for langchain-nvidia-ai-endpoints from https://files.pythonhosted.org/packages/96/02/5abe603d4888549fa026ae58002c64f86383aa123a74513d4cf6cc945e67/langchain_nvidia_ai_endpoints-0.0.1-py3-none-any.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.9.1 from https://files.pythonhosted.org/packages/93/40/d3decda219ebd5410eba627601d537ec3782efbcadba308e9ce381cc0b71/aiohttp-3.9.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for langsmith<0.1,>=0.0.83 from https://files.pythonhosted.org/packages/e3/5f/739050ab18f19ad99832b290685a2e5740a05d20580d781fd3d47b1e40f8/langsmith-0.0.85-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.9 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 186.1 MB/s eta 0:00:00
  Attempting uninstall: aiohttp
    Fou

In [2]:
import getpass
import os

os.environ["NVReranker_URL"] = # make sure you visit xxx_url to spin up NVIDID Inference Microservice for reranker to obtain an url

from langchain.retrievers.document_compressors.nim_reranker import NVRerank
nv_rerank=NVRerank()


In [3]:
import getpass
import os

## API Key can be found by going to NVIDIA NGC -> AI Foundation Models -> (some model) -> Get API Code or similar.
## 10K free queries to any endpoint (which is a lot actually).

# del os.environ['NVIDIA_API_KEY']  ## delete key and reset
if os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    print("Valid NVIDIA_API_KEY already in environment. Delete to reset")
else:
    nvapi_key = getpass.getpass("NVAPI Key (starts with nvapi-): ")
    assert nvapi_key.startswith("nvapi-"), f"{nvapi_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvapi_key

NVAPI Key (starts with nvapi-):  ········


It's also helpful (but not needed) to set up [LangSmith](https://smith.langchain.com/) for best-in-class observability

In [4]:
# test run and see that you can genreate a respond successfully 
from langchain_nvidia_ai_endpoints import ChatNVIDIA
llm = ChatNVIDIA(model="mixtral_8x7b", nvidia_api_key=nvapi_key)
result = llm.invoke("tell me a joke")
print(result.content)

Of course! I'll do my best to share a joke that follows the guidelines you've provided. Here's one:

Why don't scientists trust atoms?

Because they make up everything!

I hope this brings a smile to your face. 😊


In [5]:
import requests
query_url=os.environ["NVReranker_URL"]
query = "which way should i go?"
passages = [
    "two roads diverged in a yellow wood, and sorry i could not travel both and be one traveler, long i stood and looked down one as far as i could to where it bent in the undergrowth;",
    "then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,",
    "and both that morning equally lay in leaves no step had trodden black. oh, i marked the first for another day! yet knowing how way leads on to way i doubted if i should ever come back.",
    "i shall be telling this with a sigh somewhere ages and ages hense: two roads diverged in a wood, and i, i took the one less traveled by, and that has made all the difference."
]

request = {
  "model": "ignored",
  "query": {"text": query},
  "passages": [{"text": passage} for passage in passages]
}

response = requests.post(query_url, json=request)
rankings = response.json()["rankings"] # list of {"index": int, "score": float} with length equal to passages

print(f"high scoring passage: {passages[rankings[0]['index']]}")
print(f"low scoring passage: {passages[rankings[-1]['index']]}")

high scoring passage: and both that morning equally lay in leaves no step had trodden black. oh, i marked the first for another day! yet knowing how way leads on to way i doubted if i should ever come back.
low scoring passage: then took the other, as just as fair, and having perhaps the better claim because it was grassy and wanted wear, though as for that the passing there had worn them really about the same,


In [6]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings

embedder = NVIDIAEmbeddings(model="nvolveqa_40k")

# Alternatively, if you want to specify whether it will use the query or passage type
# embedder = NVIDIAEmbeddings(model="nvolveqa_40k", model_type="passage")

## Getting toy data

get toy data from NVIDIA [GenerativeAIExamples](https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/main/notebooks/toy_data/Sweden.txt
)

In [ ]:
!wget https://raw.githubusercontent.com/NVIDIA/GenerativeAIExamples/main/notebooks/toy_data/Sweden.txt

In [7]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator=" ",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)
# This is a long document we can split up.
with open("Sweden.txt") as f:
    state_of_the_union = f.read()
docs = text_splitter.create_documents([state_of_the_union])
print(docs[0])

page_content='Sweden, formally the Kingdom of Sweden, is a Nordic country located on the Scandinavian Peninsula in Northern Europe. It borders Norway to the west and north, Finland to the east, and is connected to Denmark in the southwest by a bridge–tunnel across the Öresund. At 447,425 square kilometres (172,752 sq mi), Sweden is the largest Nordic country, the third-largest country in the European Union, and the fifth-largest country in Europe. The capital and largest city is Stockholm. Sweden has a total population of 10.5 million, and a low population density of 25.5 inhabitants per square kilometre (66/sq mi), with around 87% of Swedes residing in urban areas, which cover 1.5% of the entire land area, in the central and southern half of the country.\nNature in Sweden is dominated by forests and many lakes, including some of the largest in Europe. Many long rivers run from the Scandes range through the landscape, primarily emptying into the northern tributaries of the Baltic Sea. 

In [8]:

list_of_docs=[doc.page_content.strip().replace('\n','') for doc in docs]
list_of_docs[-1]

"the Nordic and Baltic countries as well as the Baltic region as a wholePublic sectorSweden.se — Sweden's official portalThe Swedish Parliament – official websiteThe Government of Sweden – official websiteThe Royal Court Archived 11 October 2016 at the Wayback Machine – official website of the Swedish MonarchyNews mediaRadio Sweden – public serviceSveriges Television (in Swedish) – public serviceDagens Nyheter (in Swedish)Svenska Dagbladet (in Swedish)The Local – Sweden's news in English – independent English language news siteTradeWorld Bank Summary Trade Statistics SwedenTravelVisitSweden.com – official travel and tourism website for Sweden"

In [10]:
# Load the vectorestore back.
from langchain.vectorstores import FAISS
faiss_db = FAISS.from_documents(docs,embedder)

In [11]:
retriever = faiss_db.as_retriever(search_kwargs={"k": 3})

In [12]:
from langchain.retrievers import ContextualCompressionRetriever
## initialize the reranker from NVIDIA NVrerank class which is using the microservice API call
#feed the nv_rerank into base_compressor warped in ContextualCompressionRetriever 

compression_retriever = ContextualCompressionRetriever(
    base_compressor=nv_rerank, base_retriever=retriever
)
# make a query to test
compressed_docs = compression_retriever.get_relevant_documents(
    "Did Sweden participated in World War II?"
)


In [13]:
for d in compressed_docs:
    print(d.metadata, d.page_content )
    print('---'*10)

{'relevance_score': 1.0} World War II, although its neutrality during World War II has been disputed. Sweden was under German influence for much of the war, as ties to the rest of the world were cut off through blockades. The Swedish government felt that it was in no position to openly contest Germany, and therefore made some concessions. Sweden also supplied steel and machined parts to Germany throughout the war. The Swedish government unofficially supported Finland in the Winter War and the Continuation War by allowing volunteers and materiel to be shipped to Finland. However, Sweden supported Norwegian resistance against Germany, and in 1943 helped rescue Danish Jews from deportation to Nazi concentration camps.
During the last year of the war, Sweden began to play a role in humanitarian efforts, and many refugees, among them several thousand Jews from Nazi-occupied Europe, were rescued thanks to the Swedish rescue missions to internment camps and partly because Sweden served as a h

## Chaining

You can also easily combine with a prompt template for easy structuring of user input. We can do this using [LCEL](/docs/expression_language)

In [14]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer solely based on the following context:\n<Documents>\n{context}\n</Documents>",
        ),
        ("user", "{question}"),
    ]
)


chain = (
    {"context": compression_retriever.get_relevant_documents, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

chain.invoke("Tell me about Sweden's geographic information?")

"Sweden is a country located in northern Europe, known for its extensive forest coverage which accounts for a significant portion of its land area. The country has a varied landscape, with the highest population density found in the southern region known as the Öresund Region, along the western coast up to central Bohuslän, and in the valley of lake Mälaren and Stockholm.\n\nSweden is home to several large islands and lakes, with Gotland and Öland being the largest islands, and Vänern and Vättern being the largest lakes. Vänern is particularly noteworthy as it is the third largest lake in Europe, after Lake Ladoga and Lake Onega in Russia. When combined with the third- and fourth-largest lakes Mälaren and Hjälmaren, these lakes take up a significant part of southern Sweden's area.\n\nSweden's waterway availability is extensive, particularly in the southern part of the country. This was exploited in the 19th century with the building of the Göta Canal, which shortened the potential dist